In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:

def merge_datasets(path):	#takes the path and returns a merged dataset
    data_m = pd.read_csv(path+'movies.csv')
    data_r = pd.read_csv(path+'ratings.csv')
    data_u = pd.read_csv(path+'users.csv')

    movie_ratings = pd.merge(data_m, data_r)
    lens = pd.merge(movie_ratings, data_u)
    #print(lens.head())

    #most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
    #print(most_rated)

    return lens

def create_user_item_pair(data):
    ## picks top k rated movies & top k users with most ratings
    ## selects subset containing only above users and movies
    ## creates (user, item) pair matrix with rating as cell value
    lens = data.copy()

    most_rated_users = lens.groupby('userId').size().sort_values(ascending=False)
    mru_keys = list(most_rated_users.keys())[:600]

    most_rated_movies = lens.groupby('itemId').size().sort_values(ascending=False)
    mrm_keys = list(most_rated_movies.keys())[:400]


    data = data.loc[data['userId'].isin(mru_keys) & data['itemId'].isin(mrm_keys)]
    data = data.pivot(index='userId', columns='itemId', values='rating')
    
    
    data = np.where(np.isnan(data),0, data)
    #print(data.head())
    return data

def create_matrix_I(data):
    I = data.copy()
    I = np.asarray(I)
    I = np.where(np.isnan(I), 0, 1)
    
    return I    


In [3]:
def create_user_user_similarity_matrix(data):
    uu_matrix = cosine_similarity(data)
    return uu_matrix
    
def create_prediction_matrix(A, uu_matrix):
    ## cf algorithm's predictions
    pred_matrix = np.zeros(A.shape)
    K = A.shape[0] #no. of users
    w = A.shape[1] #no. of items
    
    mean_ratings_of_users = []
    sum_of_user_similarity = []
    for i in range(K):
        mean = np.sum(A[i])/np.count_nonzero(A[i])
        mean_ratings_of_users.append(mean)
        sum_of_user_similarity.append(np.sum(uu_matrix[i]))
    for i in range(0, w): #items
        for a in range(0, K): #users
            sum = 0
            for u in range(K):
                sum += (A[u][i] - mean_ratings_of_users[u]) * uu_matrix[a][u]
            sum /= sum_of_user_similarity[a]
            
            pred_matrix[a][i] = mean_ratings_of_users[a] + sum
            
    return pred_matrix
        

def create_episode_matrix(user_item_matrix):
    episode_matrix = []
    r = user_item_matrix.shape[0]
    c = user_item_matrix.shape[1]
    for i in range(r):
        temp = []
        for j in range(c):
            if user_item_matrix[i][j] > 0:
                temp.append(user_item_matrix[i][j])
        episode_matrix.append(temp)
    return episode_matrix


def reward(j, i, episode_matrix, pred_matrix):
    ## episode_matrix contains rating of ith user, jth movie
    return episode_matrix[i][j+2] - pred_matrix[i][j]


def train_rlcf(episode_matrix, pred_matrix, alpha=0.1, discount=0.9):
    ## |S| = 10 [0.5, 5] 
    Q = np.zeros((6, 6))
    K = len(episode_matrix)
    
    for i in range(K): #users
        for j in range(len(episode_matrix[i])-2): #items
            r = reward(j, i, episode_matrix, pred_matrix)
            qi = int(episode_matrix[i][j])
            qj = int(episode_matrix[i][j+1])
            #Q[qi][qj] = Q[qi][qj] + alpha*(r + (discount * episode_matrix[i][j+1]) - Q[qi][qj])
            Q[qi][qj] = Q[qi][qj] + alpha*(r + (discount * max(episode_matrix[i])) - Q[qi][qj])
    return Q

def prediction(Q, episode_matrix, pred_matrix):
    P = np.zeros(pred_matrix.shape)
    
    K = len(episode_matrix)
    for i in range(K):
        for j in range(2, len(episode_matrix[i])):
            qi = int(episode_matrix[i][j-2])
            qj = int(episode_matrix[i][j-1])
            P[i][j] = pred_matrix[i][j] + Q[qi, qj]
    return P
            

In [5]:
def main():
    path = 'data/'
    data = merge_datasets(path)

    A = create_user_item_pair(data)
    
    I = create_matrix_I(A)
    uu_matrix = create_user_user_similarity_matrix(A)
    '''pred_matrix = create_prediction_matrix(A, uu_matrix)
    pickle_out = open('pred_matrix.pickle', 'wb')
    pickle.dump(pred_matrix, pickle_out)
    pickle_out.close()'''
    
    pickle_in = open('pred_matrix.pickle', 'rb')
    pred_matrix = pickle.load(pickle_in)
    
    episode_matrix = create_episode_matrix(A)
    
    Q = train_rlcf(episode_matrix, pred_matrix, alpha=0.000006, discount=0.65)
    
    P = prediction(Q, episode_matrix, pred_matrix) #predictor + Q
    
    print("Prediction matrix\n\n")
    print(P[0])
    
    
    

if __name__ == '__main__':
    main()

Prediction matrix


[0.         0.         3.48830042 4.45849387 2.48378958 3.33485946
 2.73680616 4.05771564 3.02423757 4.03215887 2.7794369  3.90423377
 3.95775229 3.2475773  4.76295017 4.50248724 4.35075744 3.03692147
 3.3497066  4.2993296  3.27899177 4.11664066 3.5230314  2.65095089
 4.97752534 3.16026214 3.11135529 4.34153164 3.19223201 4.53323611
 3.18586521 3.7764516  3.23762    3.33709967 3.65653003 4.0145779
 4.47135311 3.81883253 3.75704177 1.82146826 2.75475896 3.18282044
 3.77947238 2.48672961 2.78651478 3.98750689 3.03212697 2.67852144
 3.24201153 2.89041331 4.26678397 2.5676342  1.97384667 3.67697711
 4.39974161 2.21937011 4.43977596 3.35894746 2.11548455 2.9310264
 2.60347809 4.1673952  4.14984709 3.10463228 2.63469698 2.79999072
 2.14359294 3.57304427 4.95651084 2.91818047 2.7799721  4.90262952
 4.56362381 2.23055774 4.56260316 2.64934378 3.58282387 2.4625876
 2.49187037 2.55600877 3.36939041 3.02341362 3.46980321 2.77186043
 3.43586066 3.47827136 2.94009723 4.25269649 